In [1]:
%pip install -Uq transformers 

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install -q datasets

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install -Uq accelerate

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install -q evaluate

Note: you may need to restart the kernel to use updated packages.


In [5]:
import accelerate
import transformers
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
from datasets import Dataset
import numpy as np
import torch
import torch.nn.functional as F

Для решения задачи я выбрала трансформер distillbert, так как в отличии от gpt, задачей которого является генерация текста, bert лучше справляется с задачей классификации и анализа текста.

In [6]:
transformer_model = AutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.bias', 'pre_classifier.weight', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Так я датасет я уже аугментировала для решения задачи с помощью других моделей, я их сохранила в формате csv и импортировала здесь 

In [7]:
X_train = pd.read_csv('train_augmented.csv')
X_test = pd.read_csv('test.csv')

In [8]:
X_train = X_train.drop('sms', axis=1)
X_test = X_test.drop('sms', axis=1)
X_test['label'] = X_test['label'].astype(int)
X_train['label'] = X_train['label'].astype(int)

In [9]:
X_train = X_train.dropna()


Для того чтобы сделать обучением более сбалансированным (учитывая что датасет несбалансирован) я расcчитала веса для каждого класса, чтобы использовать это в процессе обучения

In [10]:
class_weights = torch.tensor(1 - (X_train["label"].value_counts().sort_index() / len(X_train))).float() 
class_weights

tensor([0.2417, 0.7583])

In [11]:
X_train = Dataset.from_pandas(X_train)
X_test = Dataset.from_pandas(X_test)

In [12]:
def tokenize_function(examples):
    return tokenizer(examples['clean_text'],  padding='max_length', truncation=True)
train_tokens = X_train.map(tokenize_function, batched=True)
test_tokens = X_test.map(tokenize_function, batched=True)

Map:   0%|          | 0/5064 [00:00<?, ? examples/s]

Map:   0%|          | 0/1115 [00:00<?, ? examples/s]

In [13]:
training_args = TrainingArguments(
    output_dir="test_outputs",                  
)

In [14]:
import evaluate
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [15]:
from torch.nn import CrossEntropyLoss
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        
        outputs = model(**inputs) 
        logits = outputs.get("logits") 
        labels = inputs.get ("labels") 
        loss = CrossEntropyLoss(weight = class_weights) 
        
        return (loss(logits, labels), outputs) if return_outputs else loss(logits, labels)

custom_trainer = CustomTrainer(
    model=transformer_model,
    args=training_args,
    train_dataset=train_tokens,
   )


In [16]:
custom_trainer.train()

Step,Training Loss
500,0.139900
1000,0.041300
1500,0.005400


TrainOutput(global_step=1899, training_loss=0.049288335585355635, metrics={'train_runtime': 13052.1347, 'train_samples_per_second': 1.164, 'train_steps_per_second': 0.145, 'total_flos': 2012444720381952.0, 'train_loss': 0.049288335585355635, 'epoch': 3.0})

In [17]:
predictions = custom_trainer.predict(test_tokens)

In [20]:
accuracy_score = compute_metrics((predictions.predictions,test_tokens['label']))
print(accuracy_score)

{'accuracy': 0.9919282511210762}


In [25]:
classification_report_trans = classification_report(test_tokens['label'], np.argmax(predictions.predictions, axis=-1))
print(classification_report_trans)

              precision    recall  f1-score   support

           0       0.99      1.00      1.00       980
           1       0.98      0.95      0.97       135

    accuracy                           0.99      1115
   macro avg       0.99      0.97      0.98      1115
weighted avg       0.99      0.99      0.99      1115



In [30]:
trainer.save_model("./new_model")
tokenizer.save_pretrained('./new_tokenizer')

('./new_tokenizer/tokenizer_config.json',
 './new_tokenizer/special_tokens_map.json',
 './new_tokenizer/vocab.txt',
 './new_tokenizer/added_tokens.json',
 './new_tokenizer/tokenizer.json')

In [31]:
my_model = AutoModelForSequenceClassification.from_pretrained("./new_model")
my_tokenizer = AutoTokenizer.from_pretrained('./new_tokenizer')

In [37]:
from transformers import TextClassificationPipeline
pipe = TextClassificationPipeline(model=my_model, tokenizer=my_tokenizer)
input_text = 'Congratulations! You’ve won a $500 Amazon gift card. Claim it here [Link]. '
pipe(input_text)

[{'label': 'LABEL_1', 'score': 0.9995879530906677}]

Вывод: лучше всех результаты показал трансформер<br>
Несмотря на это стоит отметить что на обучение трансформера ушло очень много времени, хотя другие модели выполняют задачу не сильно хуже